This exercise uses the housing data available by default on Colab. For a description of this data, see [here](https://developers.google.com/machine-learning/crash-course/california-housing-data-description). The example is losely based on Google's (excellent) course on [logistic regression](https://colab.research.google.com/notebooks/mlcc/logistic_regression.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=logisticregression-colab&hl=en#scrollTo=B5OwSrr1yIKD).

The data is stored in the CSV file `sample_data/california_housing_test.csv`. The target is in column
`median_house_value`. The goal is to use the remaining columns to estimate whether the target is above 265000 (label `1`) or below (label `0`).

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!pip install --upgrade git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-bfyewbv3
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-bfyewbv3
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=9d6440e083693bb97d7f947c1ae825dd5e3a32e5a9c87a63cda1e339f6a32c93
  Stored in directory: /tmp/pip-ephem-wheel-cache-fpymo18y/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

In [0]:
%%cu 
/** CUBLAS example adapted from http://courses.cms.caltech.edu/cs179/ */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#include <iostream>
#include <iomanip>
#include <math.h>
#include <fstream>

/* Includes, cuda */
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <helper_cuda.h>

using namespace std;

/* Matrix size */
#define data_columns  (9)
#define print_rows (10)
#define above_threshold (265000.0)

/* transform matrix index to vector offset
   Since CUDA uses column major, 
   ld = number of rows 
   Example of use: a[IDX2C(0, 1, 50)] */
#define IDX2C(i,j,ld) (((j)*(ld))+(i))

// Error handling
// taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

/* Print an NxM array from the GPU */
void print_GPU_array(float* A, int N, int M) {
    // reserve memory on host
    float* tmp = (float*) malloc(N*M*sizeof(float));
    // copy A from device to host
    gpuErrchk( cudaMemcpy(tmp, A, N*M * sizeof(float), cudaMemcpyDeviceToHost) );
    //cublasGetVector(N*M, sizeof(float), A, 1, tmp, 1);
    // print array
    for (int i = 0;i<N;++i) {
      for (int j = 0;j<M;++j) {
        printf("%f ",tmp[IDX2C(i,j,N)]);
      }
      printf("\n");
    }
    free(tmp);
}

// generate random numbers in interval [min,max]
float float_rand( float min, float max )
{
    float scale = rand() / (float) RAND_MAX; /* [0, 1.0] */
    return min + scale * ( max - min );      /* [min, max] */
}


/* Read a csv file with a given number of rows and columns */
void read_csv(const char* filename, float* data_array,size_t nbrow,size_t nbcol) {
  string row_as_string;
  string value;
  double ioTemp;
  ifstream infile;
  infile.open(filename, ifstream::in);
  size_t row_count = 0;
	if (infile.is_open())
  {
      // read the headers (and discard)
			getline(infile, row_as_string, '\n');
      cout << "headers: " << row_as_string << "!" << std::endl;
      for(int i = 0; i < nbrow; i++){
  			getline(infile, row_as_string, '\n');
        // cout << "read line " << row_as_string << "!" << std::endl;
				istringstream line_stream(row_as_string);
			  for(int j = 0; j < nbcol; j++){
          getline(line_stream, value, ',');
					ioTemp = strtod(value.c_str(), NULL); 
          // cout << "("<<i<<","<<j<<") = "<< ioTemp << std::endl;

					data_array[IDX2C(i,j,nbrow)] = ioTemp;

				}
        ++row_count;
			}
		infile.close();
    cout << "Read " << row_count << " rows." << std::endl;
	}
	else cout << "Cannot open file." << endl;
}

/* Allocate memory and fill with given number of rows from housing file */
void read_housing_csv(float** data_array, size_t nbrows) {
    *data_array = (float *)malloc(nbrows * data_columns * sizeof(float));
    
    read_csv("sample_data/california_housing_test.csv",*data_array,nbrows,data_columns);

    cout << "Data (first "<<print_rows<<" rows):" << std::endl;
    // Show some entries for double checking
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < data_columns; j++){
				cout << (*data_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}

/* Split data into inputs and labels. Allocated memory for inputs and labels.
   Since cuBLAS is column major, each input is in a column.
   We also add 1.0 as first element to each input vector.
*/
void get_inputs_and_labels(float* data_array, float** input_array, float** label_array, size_t nbrows, size_t nbcols, size_t* nb_inputs, size_t* nb_labels ) {
    // The inputs are the first nbrows-1 columns.
    // The labels are the last column (index nbrows-1), booleanized
    // by the condition >= above_threshold
    size_t nbcols_input = nbcols-1+1; // remove one column, add one for 1.0
    size_t nbcols_labels = 2;
    *nb_inputs = nbcols_input;
    *nb_labels = nbcols_labels;
    *input_array = (float *)malloc(nbrows * nbcols_input * sizeof(float));    
    *label_array = (float *)malloc(nbrows * nbcols_labels * sizeof(float));    
    //cout << &input_array << " and "<< &label_array << " data " << data_array << std::endl;
    cout << "Allocated memory: " << nbrows << " rows, "<< nbcols_input << " columns." << std::endl;

    // Copy the data to X
    for(int i = 0; i < nbrows; i++){
      // Set the first element of each x to 1  
      (*input_array)[IDX2C(0,i,nbcols_input)] = 1.0;
      // Copy the rest of x
			for(int j = 1; j < nbcols_input; j++){
				(*input_array)[IDX2C(j,i,nbcols_input)] = data_array[IDX2C(i,j-1,nbrows)];
			}
      float median_house_value = data_array[IDX2C(i,nbcols-1,nbrows)];
      (*label_array)[IDX2C(i,0,nbrows)] = 0.0;
      (*label_array)[IDX2C(i,1,nbrows)] = 0.0;
      if (median_house_value >= above_threshold) {
        (*label_array)[IDX2C(i,0,nbrows)] = 1.0;
      } else {
        (*label_array)[IDX2C(i,1,nbrows)] = 1.0;        
      }
		}    
    
    // Show some entries for double checking
    cout << "Inputs (first "<<print_rows<<"):" << std::endl;
	  for(int j = 0; j < nbcols_input; j++){
      for(int i = 0; i < nbrows && i < print_rows; i++){
				cout << (*input_array)[IDX2C(j,i,nbcols_input)] << "\t";
			}
      cout << "\n";
		}
    cout << "Labels (first "<<print_rows<<"):" << std::endl;
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < nbcols_labels; j++){
				cout << (*label_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}

/* Variables:
  L : nb of layers
  Ml : array of size of each layer
  Wl : array of weights (matrix) for each layer
  Bl : array of biases (vectors) for each layer
  Xl : array of inputs (matrix)  for each layer; batch with one entry per column
  Zl : array of activations (matrix)  for each layer; batch with one entry per column
  Tl : array of type of each layer (0=softmax, 1=ReLU)
*/

/* Kernel to compute softmax on matrix of batch,
   taking one thread for each element:
      X = theta(Z)
   Attention: 
   X has one row more than Z; the first row of X is identical 
   to 1 by convention!

   Note: this kernel is not particularly efficient */
__global__ 
void softmax_kernel(float* X, float* Z, int nb_lgn, int nb_col) {
    const unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    const unsigned int col = i/nb_lgn;
    const unsigned int lgn = i%nb_lgn;

    if (col<nb_col && lgn<nb_lgn) {
      // exp(z_k)/sum_t exp(z_t)
      // First let x = exp(z)
      X[IDX2C(lgn+1,col,nb_lgn+1)] = exp(Z[IDX2C(lgn,col,nb_lgn)]);
      // normalize columns of x
      // using only the first thread for each col
      if (lgn == 0) {
          // Assign first row of X to 1.0 by convention
          X[IDX2C(0,col,nb_lgn+1)] = 1.0;
          float s = 0.0;
          for (int j = 0; j<nb_lgn; ++j) {
             s += X[IDX2C(j+1,col,nb_lgn+1)];
          }
          // divide all elements
          for (int j = 0; j<nb_lgn; ++j) {
             X[IDX2C(j+1,col,nb_lgn+1)] /= s;
          }
      }
    }
}

/* Kernel to compute relu on matrix of batch,
   taking one thread in the block for each element:
      X = theta(Z)
   Attention: 
   X has one row more than Z; the first row of X is identical 
   to 1 by convention!
*/
__global__ 
void relu_kernel(float* X, float* Z, int nb_lgn, int nb_col) {
    const unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    const unsigned int col = i/nb_lgn;
    const unsigned int lgn = i%nb_lgn;

    if (col<nb_col && lgn<nb_lgn) {
      X[IDX2C(lgn+1,col,nb_lgn+1)] = Z[IDX2C(lgn,col,nb_lgn)] < 0.0 ? 0.0 : Z[IDX2C(lgn,col,nb_lgn)];
      if (lgn == 0) {
          // Assign first row of X to 1.0 by convention
          X[IDX2C(0,col,nb_lgn+1)] = 1.0;
      }
      // printf("relu %d,%d,%f,%f\n",lgn,col,Z[IDX2C(lgn,col,nb_lgn)],X[IDX2C(lgn+1,col,nb_lgn+1)]);
    }
}

/* Kernel to compute derivative of relu on matrix of batch,
   taking one thread in the block for each element:
      Q = d/dz theta(Z)
*/
__global__ 
void deriv_relu_kernel(float* Q, float* Z, int nb_lgn, int nb_col) {
    const unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    const unsigned int col = i/nb_lgn;
    const unsigned int lgn = i%nb_lgn;

    if (col<nb_col && lgn<nb_lgn) {
      Q[IDX2C(lgn,col,nb_lgn)] = Z[IDX2C(lgn,col,nb_lgn)] < 0.0 ? 0.0 : 1.0;
      // printf("deriv relu %d,%d,%f,%f\n",lgn,col,Z[IDX2C(lgn,col,nb_lgn)],Q[IDX2C(lgn,col,nb_lgn)]);
    }
}

/*  Forward Computation by one step with batch of size B,
    assigning Z(l) and X(l)
    Compute Z(l) = W(l)^T X(l-1)
            X(l) = theta(Z(l))
*/
void forward_one(cublasHandle_t handle, int l, int* Ml, int B, float** W, float** X, float** Z, int* T) {
    // W^T(l) has dimensions M(l) by M(l-1)+1
    // X(l) has dimensions M(l)+1 by B
    // Z(l) has dimensions M(l) by B
    float alpha = 1.0;
    float beta = 0.0;
    int D = Ml[l-1]+1;
    int M = Ml[l];
    cublasStatus_t status = cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N, 
                          M, B, D, &alpha, 
                          W[l], D, X[l-1], D, &beta, Z[l], M);
    //printf("W[l]:\n");print_GPU_array(W[l],D,M);
    //printf("X[l-1]:\n");print_GPU_array(X[l-1],D,B);
    //printf("Z[l]:\n");print_GPU_array(Z[l],M,B);
    // loop over elements of Z to compute theta(Z)
    if (T[l]==0) { 
      // softmax
      softmax_kernel<<<(M*B-1)/1024+1, 1024>>>(X[l],Z[l],M,B);
    } else { // relu
      // relu
      relu_kernel<<<(M*B-1)/1024+1, 1024>>>(X[l],Z[l],M,B);     
    }
    gpuErrchk( cudaPeekAtLastError() );
    gpuErrchk( cudaDeviceSynchronize() );
    //printf("X[l]:\n");print_GPU_array(X[l],M+1,B);
}

/* Run the forward computation through the network,
   starting from a batch X0.
   The output is X[L-1]. 
   
   Note: We assume all arrays are allocated. 
*/
void forward_comp(cublasHandle_t handle, int L, int* Ml, int B, 
                  float** W, float** X, float** Z, 
                  int* T, float* X0) {
    // set X(0) = X0
    X[0] = X0;
    // from l = 1 to L, compute X(l) and Z(l)
    for (int l = 1; l<L; ++l) {
        forward_one(handle,l,Ml,B,W,X,Z,T);
    }
}

/* Computes E = X-Y.T,
   where X is taken only from the 2nd row.

   E is M by B; 
   X is M+1 by B;
   Y is M by B
*/
__global__
void back_init_kernel(float* E, const float* X, const float* Y, int M, int B) {
    const unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    const unsigned int col = i/M;
    const unsigned int lgn = i%M;

    E[IDX2C(lgn,col,M)] = X[IDX2C(lgn+1,col,M+1)] - Y[IDX2C(lgn,col,M)];
}

/* Computes the element-wise product of 
      A and theta'(Z),
   leaving out the first row of A,
   and assigns it to E.
   E is M by B, A is M+1 by B, Z is M by B.
*/
__global__
void back_combine_kernel_relu(float* E, const float* A, const float* Z, int M, int B) {
    const unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    const unsigned int col = i/M;
    const unsigned int lgn = i%M;

    E[IDX2C(lgn,col,M)] = Z[IDX2C(lgn,col,M)] < 0 ? 0.0 : A[IDX2C(lgn+1,col,M+1)];
}

/* Run the backward propagation through the network,
   starting from a batch of outputs Y.
   Y is M by B, containing one output per column.
   Note: We assume all arrays are allocated. 
*/
void back_prop(cublasHandle_t handle, int L, int* Ml, int B, 
               float** W, 
               float** X, float** Z, 
               float** A, float** E,
               int* T, const float* Y) {
    int l = L-1;
    // initialize with
    //    E(l) = X(l)-Y
    // first copy X to E, without the first row of X
    int nb = Ml[l]*B;
    back_init_kernel<<<(nb-1)/1024+1, 1024>>>(E[l],X[l],Y,Ml[l],B);

    // from l = L to 1, compute 
    //    A(l) = W(l) E(l)   // the first row here comes from the bias term
    //    E(l-1) = combine( A(l) , Z(l-1) )
    for (; l>1; --l) {
        float alpha = 1.0;
        float beta = 0.0;
        int D = Ml[l-1]+1;
        int M = Ml[l];
        cublasStatus_t status = cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, 
                              D, B, M, &alpha, 
                              W[l], D, E[l], M, &beta, A[l], D);
        back_combine_kernel_relu<<<(nb-1)/1024+1, 1024>>>(E[l-1],A[l],Z[l-1],Ml[l-1],B);
    }
}

/* Run one step of the gradient descent backwards through the
   network, using the update
      W[l] = W[l] - r/B X[l-1] E[l]^T,
   where r is the learning rate.

   W[l] is M[l-1]+1 by M[l]
   X[l-1] is M[l-1]+1 by B
   E[l] is M[l] by B
*/
void gradient_step(cublasHandle_t handle, int L, int* Ml, int B, 
               float** W, 
               float** X, float** E, float r) {
    int l = L-1;
    for (; l>=1; --l) {
        float alpha = -r/(float)B;
        float beta = 1.0;
        int D = Ml[l-1]+1;
        int M = Ml[l];
        cublasStatus_t status = cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_T, 
                              D, M, B, &alpha, 
                              X[l-1], D, E[l], M, &beta, W[l], D);
    }
}

/* Allocate memory for the network,
   given L (number of layers),
         M (size of each layer),
         B (batch size).
   This assigns addresses to the elements of the
   (already existing arrays) W,X,Z. */
void allocate_network_memory(int L, int* M, int B, 
                             float** W, 
                             float** X, float** Z,
                             float** A, float** E) {
    // go through each layer and reserve the appropriate size
    // for l=0, we need only X
    printf("allocating %d for X0\n",(M[0]+1) * B * sizeof(float));
    cudaMalloc((void **)&X[0], (M[0]+1) * B * sizeof(float));

    for (int l = 1; l < L; ++l) {
      // W(l) has dimensions M(l-1)+1 by M(l) 
      // X(l),A(l) have dimensions M(l)+1 by B
      // Z(l),E(l) have dimensions M(l) by B
      printf("allocating %d b for W(%d)\n",(M[l-1]+1) * M[l] * sizeof(float),l);
      printf("allocating %d b for X(%d)\n",(M[l]+1) * B * sizeof(float),l);
      printf("allocating %d b for Z(%d)\n",M[l] * B * sizeof(float),l);
      cudaMalloc((void **)&W[l], (M[l-1]+1) * M[l] * sizeof(float));
      cudaMalloc((void **)&X[l], (M[l]+1) * B * sizeof(float));
      cudaMalloc((void **)&Z[l], M[l] * B * sizeof(float));
      cudaMalloc((void **)&A[l], (M[l]+1) * B * sizeof(float));
      cudaMalloc((void **)&E[l], M[l] * B * sizeof(float));
    }
}

/* Main */
int main(int argc, char **argv)
{
    
    float *alldata = 0;
    size_t N = 5000; // number of data points (Google: 17000)
    size_t N_train = 2000; // points for training (Google: 12000)
    size_t N_test = N-N_train; // points for validation
    read_housing_csv(&alldata,N);
    float* X = 0; 
    float* Y = 0;
    size_t D; // number of features
    size_t M; // number of labels
    get_inputs_and_labels(alldata,&X,&Y,N,data_columns,&D,&M);
    /* Inputs and labels are now available in X and Y.
       Each input is a column in X; X is D x N
       each label is a row in Y; Y is N x M
    */
    
    int nb_iter = 200;
    int periods = nb_iter;
    float step_size = 1.0*1e-10;
    
    
    cublasStatus_t status;
    float *h_Z;
    float *h_q;
    float *d_X = 0;
    float *d_W = 0;
    float *d_Z = 0;
    float *d_q = 0;
    float *d_YT = 0;
    float alpha = 1.0f;
    float beta = 0.0f;
    size_t nX = D * N; // nb of elements in X
    size_t nW = D * M; // nb of elements in W
    size_t nZ = M * N; // nb of elements in Z=W^TX
    int i,j,k;
    float logloss;
    cublasHandle_t handle;

    status = cublasCreate(&handle);

    /* Allocate host memory for the matrices */
    h_Z = (float *)malloc(nZ * sizeof(h_Z[0]));
    h_q = (float *)malloc(M * sizeof(h_q[0]));

    float* YT = 0; // transpose of Y
    YT = (float *)malloc(N * M * sizeof(YT[0]));
    // assign to YT the transpose of Y
    for (int i = 0; i < N; ++i) {
      for (int j = 0; j < M; ++j) {
          YT[IDX2C(j,i,M)] = Y[IDX2C(i,j,N)];
      }
    }

    /* Initialize Weight Matrix 
       its dimension is nb_outputs * nb_inputs+1
    */
    float *h_W;
    h_W = (float *)malloc(nW * sizeof(h_W[0]));
    for (i = 0; i < nW  ; ++i) {
        h_W[i] = 2e-3 * 1/sqrt(N) * float_rand( -1, 1 );
    }

    /* Allocate device memory for the matrices */
    cudaMalloc((void **)&d_Z, nZ * sizeof(d_Z[0]));
    cudaMalloc((void **)&d_X, nX * sizeof(d_X[0]));
    cudaMalloc((void **)&d_W, nW * sizeof(d_W[0]));
    cudaMalloc((void **)&d_q, M * sizeof(d_q[0]));
    cudaMalloc((void **)&d_YT, N * M * sizeof(d_YT[0]));

    /* Initialize the device matrices with the host matrices */
    status = cublasSetVector(nX, sizeof(X[0]), X, 1, d_X, 1);
    status = cublasSetVector(nW, sizeof(h_W[0]), h_W, 1, d_W, 1);
    status = cublasSetVector(N*M, sizeof(YT[0]), YT, 1, d_YT, 1);


    // structure of the network
    int L = 2; // nb of layers
    int Ml[L]; // array of size of each layer
    Ml[0] = D-1;
    Ml[1] = M; 
    int B = 512; // batch size
    int T[L] = { -1, 0 }; // Activation type: none=-1, softmax=0
    // create arrays for matrices
    float* Wl[L];
    float* Xl[L];
    float* Zl[L];
    float* Al[L];
    float* El[L];

    // allocate memory for the network
    allocate_network_memory(L, Ml, B, Wl, Xl, Zl, Al, El);

    // use existing weights from above
    Wl[1] = d_W;

    // reserve memory for getting the results back */
    printf("allocating %d b for P\n",(Ml[L-1]+1)*B * sizeof(float));
    float* P_out = (float*) malloc((Ml[L-1]+1)*B * sizeof(float));
    float* Y_batch = (float*) malloc(Ml[L-1]*B * sizeof(float));

    int batch_index = 0;

    while (batch_index+B<=N) {
        printf("treating batch at index %d\n",batch_index);

      // forward computation, using existing weights Wl,
      // computing Xl,Zl along the way
      forward_comp(handle, L, Ml, B, Wl, Xl, Zl, T, &d_X[IDX2C(0,batch_index,D)]);

      // backward propagation, using Wl, Xl, Zl,
      // computing Al, El along the way
      back_prop(handle, L, Ml, B, Wl, Xl, Zl, Al, El, T, &d_YT[IDX2C(0,batch_index,M)]);

      // take a step in gradient descent, using Xl, El,
      // modifying Wl
      gradient_step(handle, L, Ml, B, Wl, Xl, El, step_size);
      printf("W[1]:\n");print_GPU_array(Wl[1],D,M);

      // get the result to the host
      status = cublasGetVector((Ml[L-1]+1)*B, sizeof(float), Xl[L-1], 1, P_out, 1);

      // take the argmax per element, ignoring the first row (which is 1.0 by convention)
      int true_class = 0;
      int nb_tested = 0;
      for (int i=0;i<B;++i) { 
          // for each element in the batch, take the argmax
          // over rows 1 to Ml[L-1]
          float pmax = 0.0;
          int fmax = 0;
          for (int j=1;j<=Ml[L-1];++j) {
              Y_batch[IDX2C(i,j-1,B)] = 0.0;
              if (P_out[IDX2C(j,i,Ml[L-1]+1)] > pmax) {
                pmax = P_out[IDX2C(j,i,Ml[L-1]+1)];
                fmax = j-1;
              }
          }
          Y_batch[IDX2C(i,fmax,B)] = 1.0;
          // compare prediction to label
          if (Y[IDX2C(i+batch_index,fmax,N)]>=0.5f) {
              ++true_class;
          }
          ++nb_tested;
      }

      if (0) {
        // print the result
        for (int i=0;i<B;++i) { 
            for (int j=1;j<=Ml[L-1];++j) {
                printf("p_%d=%f,y_%d=%f ",j-1,P_out[IDX2C(j,i,Ml[L-1]+1)],j-1,Y[IDX2C(i+N_train,j-1,N)]);
            }
            printf("\n");
        }
      }
      
      printf("Correct results: %d out of %d\n",true_class,nb_tested);
      printf("Accuracy: %f\n",(float)true_class/(float)nb_tested);

      batch_index += B;
    }
    printf("Finished.\n");

    /**************************************************
     * TO DO:
     * write a function, to clean up the memory of the
     * network
     **************************************************

    /* Memory clean up */
    free(h_Z);
    free(h_W);
    free(h_q);

    cudaFree(d_q);
    cudaFree(d_X);
    cudaFree(d_W);
    cudaFree(d_Z);

    /* Shutdown */
    status = cublasDestroy(handle);
}


headers: "longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"!
Read 5000 rows.
Data (first 10 rows):
-122.05	37.37	27	3885	661	1537	606	6.6085	344700	
-118.3	34.26	43	1510	310	809	277	3.599	176500	
-117.81	33.78	27	3589	507	1484	495	5.7934	270500	
-118.36	33.82	28	67	15	49	11	6.1359	330000	
-119.67	36.33	19	1241	244	850	237	2.9375	81700	
-119.56	36.51	37	1018	213	663	204	1.6635	67000	
-121.43	38.63	43	1009	225	604	218	1.6641	67000	
-120.65	35.48	19	2310	471	1341	441	3.225	166900	
-122.84	38.4	15	3080	617	1446	599	3.6696	194400	
-118.02	34.08	31	2402	632	2830	603	2.3333	164200	
Allocated memory: 5000 rows, 9 columns.
Inputs (first 10):
1	1	1	1	1	1	1	1	1	1	
-122.05	-118.3	-117.81	-118.36	-119.67	-119.56	-121.43	-120.65	-122.84	-118.02	
37.37	34.26	33.78	33.82	36.33	36.51	38.63	35.48	38.4	34.08	
27	43	27	28	19	37	43	19	15	31	
3885	1510	3589	67	1241	1018	1009	2310	3080	2402	
661	310	507	15	244	213	225	471	

Experimental:

In [0]:
!shuf sample_data/california_housing_test.csv > sample_data/california_housing_test_shuffled.csv

In [0]:
!wc sample_data/california_housing_train.csv

  17001   17001 1706430 sample_data/california_housing_train.csv


In [0]:
!ls -la sample_data

total 55812
drwxr-xr-x 1 root root     4096 Mar  6 10:15 .
drwxr-xr-x 1 root root     4096 Mar  3 18:11 ..
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Mar  3 18:11 california_housing_test.csv
-rw-r--r-- 1 root root   301141 Mar  6 10:15 california_housing_test_shuffled.csv
-rw-r--r-- 1 root root  1706430 Mar  3 18:11 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Mar  3 18:11 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Mar  3 18:11 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md
